In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from classes import Word2VecModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from functions import *
model_w2v_settings = return_best_model()

In [13]:
model_w2v = Word2VecModel(model_w2v_settings)

In [14]:
# read preprocessed data from pickle file
df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

,title,is_clickbait
0,"[house, dem, aide, even, see, comey, letter, j...",1
1,"[flynn, hillary, clinton, big, woman, campus, ...",0
2,"[truth, might, get, fired]",1
3,"[fifteen, civilian, killed, single, usa, airst...",1
4,"[iranian, woman, jailed, fictional, unpublishe...",1


In [15]:
# import models
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier


title_vectors = [get_word_vectors(model_w2v, title, aggregation='mean') for title in df['title']]

X = np.vstack(title_vectors)
y = df['is_clickbait'].values

In [16]:
# split data into train and test stratified by y
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42, shuffle=True)

# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# save scaled data as csv if data does not exist yet
if not os.path.exists('data/X_train.csv'):
    np.savetxt('data/X_train.csv', X_train_scaled, delimiter=',')
    np.savetxt('data/X_test.csv', X_test_scaled, delimiter=',')
    np.savetxt('data/y_train.csv', y_train, delimiter=',')
    np.savetxt('data/y_test.csv', y_test, delimiter=',')

In [17]:
# train sample model for now
classifier = CatBoostClassifier()
classifier.fit(X_train_scaled, y_train)

# predict on test data
y_pred = classifier.predict(X_test_scaled)

# calculate auc and f1
from sklearn.metrics import roc_auc_score, f1_score
print('AUC: ', roc_auc_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))


Learning rate set to 0.059137
0:	learn: 0.6689674	total: 39.4ms	remaining: 39.4s
1:	learn: 0.6493653	total: 68ms	remaining: 34s
2:	learn: 0.6312805	total: 92.6ms	remaining: 30.8s
3:	learn: 0.6151675	total: 118ms	remaining: 29.4s
4:	learn: 0.6011406	total: 142ms	remaining: 28.4s
5:	learn: 0.5886360	total: 166ms	remaining: 27.5s
6:	learn: 0.5777125	total: 188ms	remaining: 26.7s
7:	learn: 0.5673462	total: 212ms	remaining: 26.2s
8:	learn: 0.5579309	total: 236ms	remaining: 26s
9:	learn: 0.5499820	total: 259ms	remaining: 25.7s
10:	learn: 0.5426929	total: 281ms	remaining: 25.3s
11:	learn: 0.5357160	total: 303ms	remaining: 24.9s
12:	learn: 0.5295410	total: 326ms	remaining: 24.7s
13:	learn: 0.5238421	total: 348ms	remaining: 24.5s
14:	learn: 0.5184842	total: 369ms	remaining: 24.3s
15:	learn: 0.5138664	total: 391ms	remaining: 24.1s
16:	learn: 0.5092610	total: 414ms	remaining: 23.9s
17:	learn: 0.5049421	total: 437ms	remaining: 23.8s
18:	learn: 0.5009991	total: 460ms	remaining: 23.8s
19:	learn: 0.4

In [18]:
%run functions.py

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [19]:
chat_clickbait_examples = [
    "You won't believe what this celebrity did!",
    "Shocking secrets revealed about the latest trend!",
    "This simple trick will change your life forever!",
    "Is this the craziest video on the internet?",
    "10 unbelievable facts that will blow your mind!",
    "You'll never guess what happened next!",
    "Exclusive insider information on the hottest topic!",

]


for example in chat_clickbait_examples:
    print(f"{example} \nCHANCE OF CLICKBAIT: {predict_on_text(classifier, model_w2v, example)[0][1]} \n")


You won't believe what this celebrity did! 
CHANCE OF CLICKBAIT: 0.9062393918071735 

Shocking secrets revealed about the latest trend! 
CHANCE OF CLICKBAIT: 0.6543984370721739 

This simple trick will change your life forever! 
CHANCE OF CLICKBAIT: 0.7741490266896761 

Is this the craziest video on the internet? 
CHANCE OF CLICKBAIT: 0.617405762957752 

10 unbelievable facts that will blow your mind! 
CHANCE OF CLICKBAIT: 0.8687403647110127 

You'll never guess what happened next! 
CHANCE OF CLICKBAIT: 0.9460445517373255 

Exclusive insider information on the hottest topic! 
CHANCE OF CLICKBAIT: 0.3405405316071899 



In [20]:
non_clickbait_titles = [
    "Wikinews interviews Tatton Spiller, founder of political news service Simple Politics",
    "NASA's OSIRIS-REx arrives in Houston, US after returning asteroid samples to Earth"]

for example in non_clickbait_titles:
    print(f"{example} \nCHANCE OF CLICKBAIT: {predict_on_text(classifier, model_w2v, example)[0][1]} \n")



Wikinews interviews Tatton Spiller, founder of political news service Simple Politics 
CHANCE OF CLICKBAIT: 0.19511630878526762 

NASA's OSIRIS-REx arrives in Houston, US after returning asteroid samples to Earth 
CHANCE OF CLICKBAIT: 0.1425134386333806 

